In [ ]:
%%capture
%pip install wandb

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%%capture
!unzip "/content/drive/MyDrive/datasets/celeba.zip" -d "/content"


In [ ]:
%cd drive/MyDrive/
%rm -rf H-VAE
!git clone https://github.com/nderus/H-VAE

In [ ]:
%cd H-VAE

In [ ]:
%pip install wandb
import wandb

In [ ]:
import os
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras import backend
import wandb
from wandb.keras import WandbCallback
from tensorflow.keras import regularizers


In [ ]:
from encoders import EncoderResNet18, EncoderResNet34, EncoderResNet50, encoderCNN, EncoderMixNet18
from decoders import DecoderResNet18, DecoderResNet34, DecoderResNet50, decoderCNN
from datasets import data_loader
from embeddings import embedding
from reconstructions import reconstructions
from generations import Generations
from activations import VisualizeActivations
from gradcam import GradCam
from src.CVAE import CVAE

# import importlib
# importlib.reload(embeddings)
# from embeddings import embedding

backend.clear_session()

In [ ]:
# TO DO: this should be passed as arguments
dataset_name = 'celeba'
model_name = 'CVAE_2stage'
kl_coefficient = 4
encoded_dim = 64
second_dim = 4096
learning_rate = 0.0001 
learning_rate2 = .00005
epoch_count = 70
epoch_count2 = 140
batch_size = 100
patience = 5
lr_epochs = 60
lr_fac = .5
lr_epochs2 = 120


In [ ]:
#TO DO: move datasets in the repo and change root_folder

train_x, test_x, val_x, train_y, test_y, val_y, train_y_one_hot, test_y_one_hot, val_y_one_hot, input_shape, category_count, labels = data_loader(name=dataset_name,
                                                                                                                                     root_folder='/content/')

In [ ]:
kl_coefficient = (2*input_shape[1]/encoded_dim)**2
kl_coefficient

In [ ]:
%env "WANDB_NOTEBOOK_NAME" "train.ipynb"

In [ ]:
!wandb login 90cad85edf9c1ee3359c6432a50a42a974f2eb0d --relogin

In [ ]:

wandb.init(project=dataset_name, entity="nrderus",
  config = {
  "dataset": dataset_name,
  "model": model_name,
  "encoded_dim": encoded_dim,
  "kl_coefficient": kl_coefficient,
  "learning_rate": learning_rate,
  "epochs": epoch_count,
  "batch_size": batch_size,
  "patience": patience,
})

In [ ]:
if 'resnet' in model_name:
    encoder = EncoderMixNet18(encoded_dim = encoded_dim)
    encoder = encoder.model(input_shape=(input_shape[0], input_shape[1], input_shape[2] + category_count))

else:
    encoder = encoderCNN(input_shape, category_count, encoded_dim,  regularizer=regularizers.L2(.001))
    encoder = encoder.model(input_shape=(input_shape[0], input_shape[1], input_shape[2] + category_count))

encoder.summary()

In [ ]:

encoder.summary()

In [ ]:
if 'resnet' in model_name:
    decoder = DecoderResNet18( encoded_dim = encoded_dim, final_stride = 2)
    decoder = decoder.model(input_shape=(encoded_dim + category_count,))
else:
    decoder = decoderCNN(input_shape, category_count, encoded_dim, final_stride = 1, regularizer=regularizers.L2(.001))

decoder.summary()

In [ ]:
try:
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver('grpc://' + os.environ['COLAB_TPU_ADDR'])
    # This is the TPU initialization code that has to be at the beginning.
    tf.tpu.experimental.initialize_tpu_system(resolver)
    print("All devices: ", tf.config.list_logical_devices('TPU'))

    strategy = tf.distribute.experimental.TPUStrategy(resolver)
    with strategy.scope():
        cvae = CVAE(encoder, decoder, kl_coefficient, input_shape, category_count)
        cvae.built = True
        cvae_input = cvae.encoder.input[0]
        cvae_output = cvae.decoder.output
        mu = cvae.encoder.get_layer('mu').output
        log_var = cvae.encoder.get_layer('log_var').output

        def scheduler(epoch, learning_rate, lr_fac=0.5, lr_epochs=60 ):
            return learning_rate * tf.math.pow(lr_fac, tf.math.floor(float(epoch) / float(lr_epochs)))
            
        opt = keras.optimizers.Adam(learning_rate = learning_rate)
        cvae.compile(optimizer = opt, run_eagerly=False)
except:
    from src.CVAE import CVAE_balancing
    cvae = CVAE_balancing(encoder, decoder, kl_coefficient, input_shape, category_count)
    cvae.built = True
    cvae_input = cvae.encoder.input[0]
    cvae_output = cvae.decoder.output
    mu = cvae.encoder.get_layer('mu').output
    log_var = cvae.encoder.get_layer('log_var').output

    def scheduler(epoch, learning_rate, lr_fac=0.5, lr_epochs=60 ):
        return learning_rate * tf.math.pow(lr_fac, tf.math.floor(float(epoch) / float(lr_epochs)))
    opt = keras.optimizers.Adam(learning_rate = learning_rate)
    cvae.compile(optimizer = opt, run_eagerly=False)

In [ ]:
# early_stop = keras.callbacks.EarlyStopping(monitor='val_loss',
#              patience=patience, restore_best_weights=False)

lr_decay = tf.keras.callbacks.LearningRateScheduler(scheduler(learning_rate= learning_rate, lr_fac = lr_fac, lr_epochs = lr_epochs))


history = cvae.fit([train_x, train_y_one_hot],
                   validation_data = ([val_x, val_y_one_hot],None),
                   epochs = epoch_count,
                   batch_size = batch_size,
                   callbacks=[lr_decay, WandbCallback(save_model = False) ]) #save_weights_only -> ValueError: Unable to create dataset (name already exists)

In [ ]:
tf.print(cvae.gamma_x)


In [ ]:
# tf.saved_model.save(cvae.encoder, 'cvae_encoder')
# tf.saved_model.save(cvae.decoder, 'cvae_decoder')

In [ ]:
_, input_label_train, train_input = cvae.conditional_input([train_x[:1000], train_y_one_hot[:1000]])
_, input_label_test, test_input = cvae.conditional_input([test_x[:1000], test_y_one_hot[:1000]])
_, input_label_val, val_input = cvae.conditional_input([val_x[:1000], val_y_one_hot[:1000]])

train_x_mean, train_log_var = cvae.encoder.predict(train_input)
test_x_mean, test_log_var = cvae.encoder.predict(test_input)
val_x_mean, val_log_var = cvae.encoder.predict(val_input)

In [ ]:
from src.CVAE import SecondStage
from encoders import encoder2
from decoders import decoder2
z_cond = cvae.sampling(train_x_mean, train_log_var,train_y_one_hot[:1000])
encoder_2 = encoder2(encoded_dim, category_count, second_dim=second_dim, second_depth=3 )
decoder_2 = decoder2(encoded_dim, category_count, second_dim=second_dim, second_depth=3 )
cvae2 = SecondStage(encoder_2, decoder_2, category_count, batch_size)
opt2 = keras.optimizers.Adam(learning_rate = learning_rate2)
cvae2.compile(optimizer = opt2, run_eagerly=False)

In [ ]:
lr_decay = tf.keras.callbacks.LearningRateScheduler(scheduler(learning_rate= learning_rate, lr_fac = lr_fac, lr_epochs = lr_epochs2))

history2 = cvae2.fit([z_cond, train_y_one_hot[:1000]],
                    validation_data = None,
                   epochs = epoch_count2,
                   batch_size = batch_size,
                   callbacks=[lr_decay, WandbCallback(save_model = False) ]))

In [ ]:
#embedding(encoded_dim, category_count, train_x_mean, test_x_mean, val_x_mean, train_y, test_y, val_y, train_log_var, test_log_var, val_log_var, labels, quantity = 1000, avg_latent=True)

In [ ]:
reconstructions(cvae, train_x, train_y, train_x_mean, train_log_var, input_label_train, labels, set = 'train')

In [ ]:
from reconstructions import reconstructions2
z_hat = cvae2.posterior(z_cond, train_y_one_hot[:1000])

reconstructions2(cvae, train_x, train_y, z_hat, labels)

In [ ]:
reconstructions(cvae, test_x, test_y, test_x_mean, test_log_var, input_label_test, labels, set = 'test')

In [ ]:
generator = Generations(cvae, encoded_dim, category_count, input_shape, labels)
generator()

In [ ]:
from generations import generations2
generator = Generations(cvae, encoded_dim, category_count, input_shape, labels, cvae2, True)
generator()

In [ ]:

activations_encoder = VisualizeActivations(cvae, cvae.encoder, test_x, test_y_one_hot)
activations_decoder = VisualizeActivations(cvae, cvae.decoder, test_x, test_y_one_hot)
activations_encoder()
activations_decoder()

In [ ]:
if 'resnet' in model_name:
    target_layer = "layer4"
else:
    target_layer = "block3_conv2"


In [ ]:
gc = GradCam(cvae, test_x, test_y_one_hot, HQ = True, target_layer = target_layer)
gc.gradcam()


In [ ]:
gc.guided_gradcam()

In [ ]:
wandb.finish(exit_code=0, quiet = True) 